# Image Classifier for Dogs and Cats

## Features:
    1. Data augmentation

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

plt.rcParams['figure.figsize'] = 15, 8

In [2]:
from keras import Input, Model
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from keras import optimizers, losses, metrics
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

Using TensorFlow backend.


In [3]:
class CatsAndDogs:
    
    def __init__(self, datadir='cats-dogs', loss='binary', metric='binary', 
                    optimizer='adam', lr=1e-4, epochs=20, image_size=150,
                    batch_size=20, classes=['cats', 'dogs'],
                    augmentation=True, plotting=True, dropout=False, regularizer=False):
        
        self.datadir = datadir
        self.loss = loss
        self.metric = metric
        self.optimizer = optimizer
        self.lr = lr
        self.epochs = epochs
        self.image_size = image_size
        self.batch_size = batch_size
        self.classes = classes
        self.augmentation = augmentation
        self.plotting = plotting
        self.dropout = dropout
        self.regularizer = regularizer
        
    
    def _load_data(self):
        
        DATADIR = os.path.join('/Volumes/Study/GitHub/data/' + self.datadir)
        self.LOGDIR = os.path.join('/Volumes/Study/GitHub/neural-nets/graphs/', self.datadir)
        
        if not os.path.exists(self.LOGDIR):
            os.mkdir(self.LOGDIR, 755)
        
        self.train_dir = os.path.join(DATADIR + 'training/')
        self.validation_dir = os.path.join(DATADIR + 'validation/')
        self.test_dir = os.path.join(DATADIR + 'test/')
        
        
    def _preprocessing(self):
        if self.augmentation:
            self.train_datagen = ImageDataGenerator(rescale=1./255,
                                                   rotation_range=40,
                                                   width_shift_range=0.2,
                                                   height_shift_range=0.2,
                                                   shear_range=0.2,
                                                   zoom_range=0.2,
                                                   horizontal_flip=True,
                                                   fill_mode='nearest')
            
            self.validation_datagen = ImageDataGenerator(rescale=1./255)
        
        else:
            self.train_datagen = ImageDataGenerator(rescale=1./255)
            self.validation_datagen = ImageDataGenerator(rescale=1./255)
            
        self.train_generator = self.train_datagen.flow_from_directory(
                                                    self.train_dir,
                                                    classes = self.classes,
                                                    target_size = (self.image_size, self.image_size),
                                                    batch_size = self.batch_size,
                                                    class_mode = 'binary')
        self.validation_generator = self.train_datagen.flow_from_directory(
                                                    self.validation_dir,
                                                    classes = self.classes,
                                                    target_size = (self.image_size, self.image_size),
                                                    batch_size = self.batch_size,
                                                    class_mode = 'binary')
        
        
    
    def model_buidling(self):
        
        self._load_data()
        self._preprocessing()
        
        ip = Input(batch_shape=(None, self.image_size, self.image_size, 3), name='input')

        x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', name='conv1')(ip)
        x = MaxPool2D(pool_size=(2, 2), name='pool1')(x)

        x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', name='conv2')(x)
        x = MaxPool2D(pool_size=(2, 2), name='pool2')(x)

        x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu', name='conv3')(x)
        x = MaxPool2D(pool_size=(2, 2), name='pool3')(x)

        x = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu', name='conv4')(x)
        x = MaxPool2D(pool_size=(2, 2), name='pool4')(x)

        x = Flatten(name='flatten')(x)
        if self.dropout:
            x = Dropout(rate=0.3, name='dropout1')(x)

        x = Dense(units=128, activation='relu', name='dense1')(x)
        if self.dropout:
            x = Dropout(rate=0.3, name='dropout2')(x)

        output = Dense(units=1, activation='sigmoid', name='output')(x)

        self.model = Model(inputs=ip, outputs=output)
        self.model.summary()
        return
    
    def model_compilation(self):
        
        self.model_buidling()
        
        if self.optimizer=='adam':
            self.opt = optimizers.adam(self.lr)
        elif self.optimizer=='rms':
            self.opt = optimizers.RMSprop(self.lr)
            
        if self.loss=='binary':
            self.losses = losses.binary_crossentropy
        elif self.loss=='multiclass':
            self.losses = losses.categorical_crossentropy
            
        if self.metric=='binary':
            self.accuracy = metrics.binary_accuracy
        elif self.metric=='mulitclass':
            self.accuracy = metrics.categorical_crossentropy
        
        self.model.compile(loss=self.losses, optimizer=self.opt, metrics=[self.accuracy])
        return
    
    def training(self):
        
        self.model_compilation()
        
        self.history = self.model.fit_generator(self.train_generator,
                             steps_per_epoch=100,
                             epochs=self.epochs,
                             verbose=1,
                             validation_data=self.validation_generator,
                             validation_steps=50)

        self.model.save(os.path.join(LOGDIR + 'cat_dog_v1_1.h5'))
    
        if self.plotting:
            
            plt.plot(self.history.history['loss'], label = 'training_loss')
            plt.plot(self.history.history['val_loss'], label = 'validation_loss')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.title('Training and Validation loss')
            plt.legend()
            plt.show()
            
            plt.plot(self.history.history['binary_accuracy'], label = 'training_accuracy')
            plt.plot(self.history.history['val_binary_accuracy'], label = 'validation_accuracy')
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy')
            plt.title('Training and Validation Accuracy')
            plt.legend()
            plt.show()
        
        return

In [4]:
model = CatsAndDogs(datadir='cats-dogs', loss='binary', metric='binary', 
                    optimizer='adam', lr=1e-4, epochs=20, image_size=150,
                    batch_size=20, classes=['cats', 'dogs'],
                    augmentation=True, plotting=True, dropout=False, regularizer=False)

In [5]:
model.training()

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 150, 150, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 150, 150, 32)      896       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 75, 75, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 75, 75, 64)        18496     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 37, 37, 64)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 37, 37, 128)       73856     
________________________________________________________________

KeyboardInterrupt: 